In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.appName("JoinExamples").getOrCreate()

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("city", StringType(), True)
])

schema2 = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("city", StringType(), True),
    StructField("local", StringType(), True)
])

data1 = [(1, "Alice", 25, "New York"), (None, "Bob", 30, "London"), (3, "Charlie", 22, "Paris"), (4, "David", 28, "Tokyo")]
df1 = spark.createDataFrame(data1, schema=schema)

data2 = [(1, "Alice", 24, "New York","monday"), (5, "Eve", 24, "Sydney",None), (1, "Bob", 35, "London","Tuesday"), (None, "Frank", 35, "Berlin",None),(None, "kil", 35, "Berlin", "sunday")]  # Note: Some overlap in 'id' and 'name'
df2 = spark.createDataFrame(data2, schema=schema2)

df1.show()
df2.show()

25/03/13 11:05:49 WARN Utils: Your hostname, codespaces-56288b resolves to a loopback address: 127.0.0.1; using 10.0.2.249 instead (on interface eth0)
25/03/13 11:05:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/13 11:05:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/13 11:06:08 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+----+-------+---+--------+
|  id|   name|age|    city|
+----+-------+---+--------+
|   1|  Alice| 25|New York|
|NULL|    Bob| 30|  London|
|   3|Charlie| 22|   Paris|
|   4|  David| 28|   Tokyo|
+----+-------+---+--------+

+----+-----+---+--------+-------+
|  id| name|age|    city|  local|
+----+-----+---+--------+-------+
|   1|Alice| 24|New York| monday|
|   5|  Eve| 24|  Sydney|   NULL|
|   1|  Bob| 35|  London|Tuesday|
|NULL|Frank| 35|  Berlin|   NULL|
|NULL|  kil| 35|  Berlin| sunday|
+----+-----+---+--------+-------+



In [3]:
# left join
df3 = df1.unionByName(df2, allowMissingColumns=True)
df3.show()

df4 = df3.dropDuplicates(["id"]).orderBy(["id"])
df4.show()

df5 = df4.distinct().orderBy(["id"])
df5.show()



+----+-------+---+--------+-------+
|  id|   name|age|    city|  local|
+----+-------+---+--------+-------+
|   1|  Alice| 25|New York|   NULL|
|NULL|    Bob| 30|  London|   NULL|
|   3|Charlie| 22|   Paris|   NULL|
|   4|  David| 28|   Tokyo|   NULL|
|   1|  Alice| 24|New York| monday|
|   5|    Eve| 24|  Sydney|   NULL|
|   1|    Bob| 35|  London|Tuesday|
|NULL|  Frank| 35|  Berlin|   NULL|
|NULL|    kil| 35|  Berlin| sunday|
+----+-------+---+--------+-------+



+----+-------+---+--------+------+
|  id|   name|age|    city| local|
+----+-------+---+--------+------+
|NULL|    Bob| 30|  London|  NULL|
|   1|  Alice| 24|New York|monday|
|   3|Charlie| 22|   Paris|  NULL|
|   4|  David| 28|   Tokyo|  NULL|
|   5|    Eve| 24|  Sydney|  NULL|
+----+-------+---+--------+------+



+----+-------+---+--------+------+
|  id|   name|age|    city| local|
+----+-------+---+--------+------+
|NULL|    Bob| 30|  London|  NULL|
|   1|  Alice| 24|New York|monday|
|   3|Charlie| 22|   Paris|  NULL|
|   4|  David| 28|   Tokyo|  NULL|
|   5|    Eve| 24|  Sydney|  NULL|
+----+-------+---+--------+------+



In [4]:

# df3.show()

df7 = df1.join(df2, df1.id.eqNullSafe(df2.id), 'left')
df7.show()

#df8 = df1.union(df2)
#df8.show()

df9 = df1.unionByName(df2,allowMissingColumns=True)
df9.show()
# right join
 
df4 = df2.join(df1, df1.id == df2.id, 'right')
# df4.show()

# inner join 
df5 = df1.join(df2, df1.id == df2.id, 'inner' )
# df5.show()

df6 = df1.join(df2, df1.id == df2.id, 'outer')
# df6.show()

+----+-------+---+--------+----+-----+----+--------+-------+
|  id|   name|age|    city|  id| name| age|    city|  local|
+----+-------+---+--------+----+-----+----+--------+-------+
|   1|  Alice| 25|New York|   1|  Bob|  35|  London|Tuesday|
|   1|  Alice| 25|New York|   1|Alice|  24|New York| monday|
|NULL|    Bob| 30|  London|NULL|  kil|  35|  Berlin| sunday|
|NULL|    Bob| 30|  London|NULL|Frank|  35|  Berlin|   NULL|
|   4|  David| 28|   Tokyo|NULL| NULL|NULL|    NULL|   NULL|
|   3|Charlie| 22|   Paris|NULL| NULL|NULL|    NULL|   NULL|
+----+-------+---+--------+----+-----+----+--------+-------+

+----+-------+---+--------+-------+
|  id|   name|age|    city|  local|
+----+-------+---+--------+-------+
|   1|  Alice| 25|New York|   NULL|
|NULL|    Bob| 30|  London|   NULL|
|   3|Charlie| 22|   Paris|   NULL|
|   4|  David| 28|   Tokyo|   NULL|
|   1|  Alice| 24|New York| monday|
|   5|    Eve| 24|  Sydney|   NULL|
|   1|    Bob| 35|  London|Tuesday|
|NULL|  Frank| 35|  Berlin|  

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Create a Spark session
spark = SparkSession.builder.appName("Employee Data").getOrCreate()

# Define the schema for employee details DataFrame
employee_details_schema = StructType([
    StructField("employee_id", IntegerType(), True),
    StructField("employee_name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("department", StringType(), True)
])

# Create sample employee details data
employee_details_data = [
    (1, "John Doe", 30, "Engineering"),
    (2, "Jane Smith", 25, "Marketing"),
    (3, "Sam Brown", 35, "Sales")
]

# Create the employee details DataFrame
employee_details_df = spark.createDataFrame(employee_details_data, employee_details_schema)

# Define the schema for employee address DataFrame
employee_address_schema = StructType([
    StructField("employee_id", IntegerType(), True),
    StructField("address", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True)
])

# Create sample employee address data
employee_address_data = [
    (1, "123 Main St", "Springfield", "IL"),
    (2, "456 Elm St", "Riverside", "CA"),
    (3, "789 Oak St", "Madison", "WI")
]

# Create the employee address DataFrame
employee_address_df = spark.createDataFrame(employee_address_data, employee_address_schema)

# Show the DataFrames
employee_details_df.show()
employee_address_df.show()


df3 = employee_details_df.join(employee_address_df, employee_details_df.employee_id == employee_address_df.employee_id, 'inner')
df3.show()


25/03/13 11:06:18 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-----------+-------------+---+-----------+
|employee_id|employee_name|age| department|
+-----------+-------------+---+-----------+
|          1|     John Doe| 30|Engineering|
|          2|   Jane Smith| 25|  Marketing|
|          3|    Sam Brown| 35|      Sales|
+-----------+-------------+---+-----------+

+-----------+-----------+-----------+-----+
|employee_id|    address|       city|state|
+-----------+-----------+-----------+-----+
|          1|123 Main St|Springfield|   IL|
|          2| 456 Elm St|  Riverside|   CA|
|          3| 789 Oak St|    Madison|   WI|
+-----------+-----------+-----------+-----+

+-----------+-------------+---+-----------+-----------+-----------+-----------+-----+
|employee_id|employee_name|age| department|employee_id|    address|       city|state|
+-----------+-------------+---+-----------+-----------+-----------+-----------+-----+
|          1|     John Doe| 30|Engineering|          1|123 Main St|Springfield|   IL|
|          2|   Jane Smith| 25|  Marke

In [12]:
!pip install delta-spark

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Create two DataFrames
data1 = [
    (1, "Alice", 25, "New York"), 
    (2, "Bob", 30, "London")
    ]
data2 = [
    (3, "Charlie", 22, "Paris", "France"),
    (4, "David", 28, "Tokyo", "Japan")
    ]

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("city", StringType(), True)
])

schema1 = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("city", StringType(), True),
    StructField("country", StringType(), True),
])

df1 = spark.createDataFrame(data1,
                             schema=schema)
df2 = spark.createDataFrame(data2,
                             schema=schema1)

df1.show()
df2.show()

# Perform union
df_union = df1.unionByName(df2,
                    allowMissingColumns=True)
df_union.show()


from delta.tables import DeltaTable

# Create a Delta table

DeltaTable.createIfNotExists(spark) \
    .tableName("employee_details") \
    .location("/tmp/delta/employee_details") \
    .data(employee_details_df) \
    .execute()


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
+---+-----+---+--------+
| id| name|age|    city|
+---+-----+---+--------+
|  1|Alice| 25|New York|
|  2|  Bob| 30|  London|
+---+-----+---+--------+

+---+-------+---+-----+-------+
| id|   name|age| city|country|
+---+-------+---+-----+-------+
|  3|Charlie| 22|Paris| France|
|  4|  David| 28|Tokyo|  Japan|
+---+-------+---+-----+-------+

+---+-------+---+--------+-------+
| id|   name|age|    city|country|
+---+-------+---+--------+-------+
|  1|  Alice| 25|New York|   NULL|
|  2|    Bob| 30|  London|   NULL|
|  3|Charlie| 22|   Paris| France|
|  4|  David| 28|   Tokyo|  Japan|
+---+-------+---+--------+-------+



TypeError: 'JavaPackage' object is not callable

In [3]:
!pip install delta-spark


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.appName("JoinExamples").getOrCreate()

from delta.tables import DeltaTable

# Create a Delta table
DeltaTable.createIfNotExists(spark) \
    .tableName("employee_details") \
    .location("/tmp/delta/employee_details") \
    .addColumns(employee_details_schema) \
    .execute()

TypeError: 'JavaPackage' object is not callable